## Resizing Image

In [1]:
import os
from PIL import Image

# Define the directory paths
input_dataset_dir = r"E:\attendance\faces"# Replace with the path to your root folder
output_dataset_dir = r"E:\attendance\resized"  # Directory where resized images will be saved

# Function to resize images
def resize_images(input_dir, output_dir, size=(160, 160)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for person in os.listdir(input_dir):
        person_dir = os.path.join(input_dir, person)
        output_person_dir = os.path.join(output_dir, person)
        
        if not os.path.exists(output_person_dir):
            os.makedirs(output_person_dir)
        
        for img_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_name)
            try:
                img = Image.open(img_path)
                img_resized = img.resize(size)
                img_resized.save(os.path.join(output_person_dir, img_name))
            except Exception as e:
                print(f"Error resizing image {img_name} for person {person}: {e}")

# Call the function to resize images
resize_images(input_dataset_dir, output_dataset_dir)


## Augmentation

In [2]:
from torchvision import transforms
from PIL import Image
import random
import os

# Define augmentation transformations
augmentation_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # Flip the image horizontally with a 50% chance
    transforms.RandomRotation(degrees=20),   # Randomly rotate the image up to 20 degrees
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Change brightness, contrast, etc.
    transforms.RandomResizedCrop(size=(160, 160), scale=(0.8, 1.0)),  # Crop and resize
])

# Augment an image and return the augmented version
def augment_image(image_path):
    img = Image.open(image_path).convert('RGB')
    augmented_img = augmentation_transforms(img)
    return augmented_img

# Augment and save images to the output directory
def augment_and_save_images(input_dir, output_dir, augment_factor=5):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    for person in os.listdir(input_dir):
        person_dir = os.path.join(input_dir, person)
        output_person_dir = os.path.join(output_dir, person)
        
        if not os.path.exists(output_person_dir):
            os.makedirs(output_person_dir)
        
        for img_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_name)
            img = Image.open(img_path)
            
            # Save original image
            img.save(os.path.join(output_person_dir, img_name))
            
            # Save augmented images
            for i in range(augment_factor):
                augmented_img = augment_image(img_path)
                augmented_img.save(os.path.join(output_person_dir, f"{img_name.split('.')[0]}_aug_{i}.jpg"))

# Augment and save dataset
input_dataset_dir =r"E:\attendance\resized"
output_dataset_dir =r"E:\attendance\aug"
augment_and_save_images(input_dataset_dir, output_dataset_dir, augment_factor=4)  # Augment each image 3 times


In [3]:
path_to_augmented_dataset = r"E:\attendance\aug"

## Loading Data

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os
import matplotlib.pyplot as plt

# Define the complete transformation (resizing, optional augmentations)
complete_transform = transforms.Compose([
    transforms.Resize((250, 250)),  # Resize to 250x250 pixels
    transforms.RandomHorizontalFlip(),  # Randomly flip images
    transforms.ToTensor(),           # Convert image to tensor
])

# Create the dataset using the dataset directory
dataset = datasets.ImageFolder(root=output_dataset_dir, transform=complete_transform)

# Create a DataLoader
data_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Check the number of classes in the dataset
num_classes = len(dataset.classes)
print(f"Number of classes: {num_classes}")

# Visualize a batch of images
def show_images(images, labels):
    plt.figure(figsize=(12, 6))
    num_images = min(len(images), 10)  # Handle fewer than 10 images
    for i in range(num_images):
        plt.subplot(2, 5, i + 1)
        plt.imshow(images[i].permute(1, 2, 0))  # Permute dimensions for visualization
        plt.title(dataset.classes[labels[i]])
        plt.axis('off')
    plt.show()

# Inspecting a batch of images
images, labels = next(iter(data_loader))
show_images(images, labels)


## Embeddings and Knn Classifier

In [ ]:
import os
import numpy as np
import joblib
import torch
from PIL import Image
from facenet_pytorch import MTCNN, InceptionResnetV1
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Initialize device, MTCNN, and InceptionResnetV1
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
mtcnn = MTCNN(keep_all=False, device=device)
model = InceptionResnetV1(pretrained='vggface2').eval().to(device)

def get_embeddings(image_path):
    """Extract embeddings from a given image path."""
    try:
        img = Image.open(image_path).convert("RGB")
        boxes, _ = mtcnn.detect(img)
        if boxes is not None:
            faces = mtcnn(img)
            faces = faces.to(device)
            embeddings = model(faces.unsqueeze(0))
            return embeddings.detach().cpu().numpy()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
    return None

def process_directory(input_dir):
    """Process images in a directory to extract embeddings."""
    embeddings_dict = {}
    
    for person in os.listdir(input_dir):
        person_dir = os.path.join(input_dir, person)
        if not os.path.isdir(person_dir):
            continue
        
        embeddings_list = []
        for img_name in os.listdir(person_dir):
            img_path = os.path.join(person_dir, img_name)
            embeddings = get_embeddings(img_path)
            if embeddings is not None:
                embeddings_list.append(embeddings)
        
        if embeddings_list:
            embeddings_dict[person] = np.vstack(embeddings_list)

    return embeddings_dict

def train_knn_classifier(embeddings_dict):
    """Train KNN classifier on the extracted embeddings."""
    X, y = [], []
    for person, embeddings in embeddings_dict.items():
        X.append(embeddings)
        y.extend([person] * embeddings.shape[0])

    X = np.vstack(X)
    y = np.array(y)

    # Split data into training and test sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    k_values = range(1, 21)
    accuracies = []

    for k in k_values:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        accuracies.append(knn.score(X_test, y_test))

# Plot the accuracy
    plt.figure(figsize=(10, 5))
    plt.plot(k_values, accuracies, marker='o')
    plt.title('K-Value vs. Accuracy')
    plt.xlabel('Number of Neighbors (k)')
    plt.ylabel('Accuracy')
    plt.xticks(k_values)
    plt.grid()
    plt.show()

    # Train KNN classifier
    knn = KNeighborsClassifier(n_neighbors=8)
    knn.fit(X_train, y_train)

    return knn, X_test, y_test

def evaluate_model(knn, X_test, y_test):
    """Evaluate the KNN model and display metrics."""
    y_pred = knn.predict(X_test)
    
    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    plt.figure(figsize=(10, 7))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=np.unique(y_test), yticklabels=np.unique(y_test))
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title(f'Confusion Matrix (Accuracy: {accuracy:.2f})')
    plt.show()

    print(classification_report(y_test, y_pred))

# Main execution
input_dir = r"E:\attendance\aug"
embeddings_dict = process_directory(input_dir)
knn, X_test, y_test = train_knn_classifier(embeddings_dict)
evaluate_model(knn, X_test, y_test)

# Save the classifier
joblib.dump(knn, 'allknn.pkl')
